In [16]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout

pd.options.mode.chained_assignment = None

In [2]:
tf.config.list_physical_devices('GPU')

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout

pd.options.mode.chained_assignment = None

df = pd.read_csv("./totolotek.csv")
# df = pd.read_csv("/content/drive/MyDrive/totolotek.csv")

df.head()
df.tail()
df.info()
df.describe()
df.drop(['GAME', 'DATE'], axis=1, inplace=True)
df.head()

scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)
transformed_df.head()
number_of_rows = df.values.shape[0]
window_length = 7
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)

for i in range(0, number_of_rows - window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

model = Sequential()

model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))

model.add(Dropout(0.2))
model.add(Dense(59))
model.add(Dense(number_of_features))

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])
model.fit(x=X, y=y, batch_size=100, epochs=200, verbose=2)

to_predict = df.tail(8)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict = np.array(to_predict)
scaled_to_predict = scaler.transform(to_predict)

y_pred = model.predict(np.array([scaled_to_predict]))
print("\nPrzewidziane numery:\t", scaler.inverse_transform(y_pred).astype(int)[0])
prediction = df.tail(1)
prediction = np.array(prediction)
print("Prawdziwe numery:\t", prediction[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6971 entries, 0 to 6970
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   GAME    6971 non-null   int64 
 1   DATE    6971 non-null   object
 2   A       6971 non-null   int64 
 3   B       6971 non-null   int64 
 4   C       6971 non-null   int64 
 5   D       6971 non-null   int64 
 6   E       6971 non-null   int64 
 7   F       6971 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 435.8+ KB
Epoch 1/200
70/70 - 12s - loss: 1.0009 - accuracy: 0.1732 - 12s/epoch - 178ms/step
Epoch 2/200
70/70 - 5s - loss: 1.0002 - accuracy: 0.1862 - 5s/epoch - 65ms/step
Epoch 3/200
70/70 - 5s - loss: 0.9997 - accuracy: 0.1827 - 5s/epoch - 65ms/step
Epoch 4/200
70/70 - 5s - loss: 0.9996 - accuracy: 0.1861 - 5s/epoch - 65ms/step
Epoch 5/200
70/70 - 5s - loss: 0.9997 - accuracy: 0.1903 - 5s/epoch - 65ms/step
Epoch 6/200
70/70 - 5s - loss: 0.9994 - accuracy: 0.2040 - 5s/epoch - 65ms/step